In [5]:
import os
import sys
import time
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from timeit import default_timer as timer
from tqdm import tqdm
from tqdm.notebook import tqdm

from collections import defaultdict,Counter

import scipy.io
import optuna
import matplotlib.pyplot as plt

import matplotlib.mlab as mlab
import matplotlib.gridspec as gridspec

import scipy as sp
import matplotlib.pyplot as plt

from Data.DPDBlocks.blocks import AFIR, Delay, Prod_cmp, ABS, Polynomial
from Data.utilits.calculate_functions import ACPR_calc, MSE, NMSE
from torch.utils.data import DataLoader


## Dataload 

In [12]:
Batch_size = 64
name = 'work/VorkutovDA/Data/BlackBoxData/BlackBoxData_80'
# name = 'BlackBoxData'
# name = '../BlackBoxData/data1'
mat = scipy.io.loadmat(name)
x = np.array(mat['x']).reshape(-1,1)/2**15
d = np.array(mat['y']).reshape(-1,1)/2**15
# x = np.array(mat['xE']).reshape(-1,1)/2**15
# d = np.array(mat['d']).reshape(-1,1)/2**15
# x, d = mat['xE'], mat['d']
x_real, x_imag = torch.from_numpy(np.real(x)), torch.from_numpy(np.imag(x))
d_real, d_imag = torch.from_numpy(np.real(d)), torch.from_numpy(np.imag(d))
X = torch.DoubleTensor(torch.cat((x_real, x_imag), dim=1)).reshape(-1,2,1)
D = torch.DoubleTensor(torch.cat((d_real, d_imag), dim=1)).reshape(-1,2,1)

train_queue = torch.utils.data.DataLoader(
    torch.cat((X,D),dim=-1), batch_size=Batch_size)#, pin_memory=True)

valid_queue = torch.utils.data.DataLoader(
    torch.cat((X,D),dim=-1), batch_size=X.shape[0])#,pin_memory=True)

In [13]:
#NewDAtaLoader


## Train Function

In [14]:
loss_fn = nn.MSELoss()

## Class Model

In [15]:
type(valid_queue )

torch.utils.data.dataloader.DataLoader

In [16]:
import os
import sys
import time
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from timeit import default_timer as timer
from tqdm import tqdm_notebook
from collections import defaultdict,Counter

import scipy.io

from Data.DPDBlocks.blocks import AFIR,ABS,Polynomial,Delay,Prod_cmp

import matplotlib.pyplot as plt

import matplotlib.mlab as mlab
import matplotlib.gridspec as gridspec


def eval_model(valid_queue, model, criterion):
    for step, (valid) in enumerate(valid_queue):
        model.eval()
        input_batch = Variable(valid[:,:,:1],requires_grad=False).permute(2,1,0).cpu()
        desired = Variable(valid[:,:,1:],requires_grad=False).permute(2,1,0).cpu()
        #out = model.forward(input_batch)
        out = sum(list(map( lambda n: n(input_batch),  model)))

        loss=criterion(out,desired)
        #draw_spectrum(input_batch,desired,out)
        accuracy = NMSE(input_batch, out - desired)

    return loss,accuracy

def train_of_epoch(train_queue, model, criterion, optimizer):
    

    
    for step, (train) in enumerate(train_queue):

        input_batch = Variable(train[:,:,:1],requires_grad=False).permute(2,1,0).cpu()
        desired = Variable(train[:,:,1:],requires_grad=False).permute(2,1,0).cpu()
        optimizer.zero_grad()
        #out = model.forward(input_batch)
        out = sum(list(map( lambda n: n(input_batch),  model)))

        loss = criterion(out, desired)
        
        
        loss.backward()
        
        optimizer.step()


def train(train_queue, valid_queue, model, criterion, optimizer,n_epoch,
          scheduler,log_every=1,save_flag=True,path_to_experiment='', dataFromEpohAccuracy = [], dataFromEpohLoss = []):
    
    
    model.train()

    min_loss=0
    hist=defaultdict(list)
    t0=timer()
    for it in tqdm (range(n_epoch)):
        model.train(True)
        train_of_epoch(train_queue, model, criterion, optimizer)
        scheduler.step()
        if it%log_every==0:
            loss_v,accuracy_v=eval_model(valid_queue, model, loss_fn)
            print('Loss = ',loss_v.cpu().detach().numpy(), 'Accuracy = ', accuracy_v.cpu().detach().numpy(), 'dbs')
            dataFromEpohAccuracy.append(accuracy_v)
            dataFromEpohLoss.append(loss_v)
            
            if save_flag:
                with open(path_to_experiment + '/hist.pkl', 'wb') as output:
                    pickle.dump(hist, output)

                    torch.save(model.state_dict(), path_to_experiment + '/model.pt')
                if hist['train_loss_db'][-1] < min_loss:
                            min_loss = hist['train_loss_db'][-1]
                            torch.save(model.state_dict(), path_to_experiment + '/best_model.pth')

#class Cell_try_2(nn.Module):
#    def __init__(self,M=15,D=0,Poly_order=8,Passthrough=False):
#        super(Cell_try_2,self).__init__()
#        self.f = AFIR(M,0)
#        self.pol = Polynomial(Poly_order,Passthrough)
#        self.prod = Prod_cmp()
#        self.delay = Delay(D)
#    def forward(self,x):
#        #return self.prod(self.f(self.delay(x)), self.pol(self.delay(x)))
#        return self.prod( self.f(x), self.pol(self.f(self.delay(x))) )
class Cell_try_2(nn.Module):
    def __init__(self,M=15,D=0,Poly_order=8,Passthrough=False):
        super(Cell_try_2,self).__init__()
        self.f = AFIR(M,0)
        self.pol = Polynomial(Poly_order,Passthrough)
        self.prod = Prod_cmp()
        self.delay = Delay(D)
    def forward(self,x):
        #return self.prod(self.f(self.delay(x)), self.pol(self.delay(x)))
        return self.prod( self.f(self.delay(x)), self.f(self.pol(self.delay(x))) )

In [23]:
D = {'p': [4, 5, 6, 7, 8, 9], 'k' : [3, 5, 7, 9], 'z' : [-2, -1, 0, 1, 2]}
ex_D = {} # extremum vals of D
step_size=10
gamma=0.1
for key in D.keys():
  ex_D[key] = [ min(D[key]), max(D[key])]

# complex reference model
ref_model = {'k': [9,5,9,7,9],'p': [9,8,6,9,6]}

### params of the functional
score_huge = -37
score_min = -20.0
complex_huge = 2 * ( sum(ref_model['k']) + sum(ref_model['p']) )
complex_min = 2 * (4 * 5 + 3 * 5)
trtr_coef = 0.4

def objective(trial):
  # create and train NN
  net = torch.nn.ModuleList()

  complex_cur = 0

  for i in range(2):
    # det hyperparams 
    poly_ord = trial.suggest_int('p'+str(i), ex_D['p'][0], ex_D['p'][1])
    conv_ord = trial.suggest_int('k'+str(i), ex_D['k'][0], ex_D['k'][1], step=2)
    net.append(Cell_try_2(M=conv_ord, D=(ex_D['z'][0] + 1*i), Poly_order=poly_ord))
    complex_cur = complex_cur + poly_ord + conv_ord

  net = net.to(torch.device('cpu'))
  optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
  train(train_queue, valid_queue, net, loss_fn, optimizer, 20, scheduler, save_flag=False)

  loss_cur, accuracy_cur = eval_model(valid_queue, net, loss_fn)
  score_cur = accuracy_cur.item()

  #return score_cur
  return  (complex_cur - complex_min) / (complex_huge - complex_min) + trtr_coef * (score_huge - score_cur) / (score_huge - score_min)


In [1]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction="minimize", pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=1)
print(study.best_params)

NameError: name 'optuna' is not defined

In [ ]:


D = {'p': [4, 5, 6, 7, 8, 9], 'k' : [3, 5, 7, 9], 'z' : [-2, -1, 0, 1, 2]}
ex_D = {} # extremum vals of D

for key in D.keys():
  ex_D[key] = [ min(D[key]), max(D[key])]
  
# complex reference model
ref_model = {'k': [9,5,9,7,9],'p': [9,8,6,9,6]}

### params of the functional
score_huge = -33.88
score_min = -20.0
complex_huge = sum(ref_model['k']) + sum(ref_model['p'])
complex_min = 4 * 5 + 3 * 5
trtr_coef = 0.4
gamma=0.95
step_size=5

COMP_2 = []
SCR_2 = []
zz = [-3,-2,-1,2,3]


net = torch.nn.ModuleList()
complex_cur = 0

#TPE
net.append(Cell_try_2(M=9, D=zz[0], Poly_order=5))
net.append(Cell_try_2(M=7, D=zz[1], Poly_order=9))
net.append(Cell_try_2(M=7, D=zz[2], Poly_order=9))
net.append(Cell_try_2(M=8, D=zz[3], Poly_order=9))
net.append(Cell_try_2(M=7, D=zz[4], Poly_order=6))



net = net.to(torch.device('cpu'))
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
net.train()

train( train_queue, valid_queue, net, loss_fn, optimizer,20,scheduler,save_flag=False)

loss_cur, accuracy_cur = eval_model(valid_queue, net, loss_fn)
score_cur = accuracy_cur.item()

COMP_2.append(complex_cur)
SCR_2.append(score_cur)

print( score_cur)





In [ ]:
#TPESampler
netTPE = torch.nn.ModuleList()
complex_cur = 0
COMP_netTPE = []
SCR_netTPE = []
dataFromEpohAccuracyTPE = []
dataFromEpohLossTPE = []

netTPE.append(Cell_try_2(M=9, D=zz[0], Poly_order=5))
netTPE.append(Cell_try_2(M=7, D=zz[1], Poly_order=9))
netTPE.append(Cell_try_2(M=7, D=zz[2], Poly_order=9))
netTPE.append(Cell_try_2(M=9, D=zz[3], Poly_order=8))
netTPE.append(Cell_try_2(M=7, D=zz[4], Poly_order=6))


netTPE = netTPE.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netTPE.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netTPE.train()

train( train_queue, valid_queue, netTPE, loss_fn, optimizer,50,scheduler,save_flag=False, dataFromEpohAccuracy = dataFromEpohAccuracyTPE, dataFromEpohLoss =dataFromEpohLossTPE )

loss_cur, accuracy_cur = eval_model(valid_queue, netTPE, loss_fn)
score_cur = accuracy_cur.item()

COMP_netTPE.append(complex_cur)
SCR_netTPE.append(score_cur)

print( score_cur)

In [ ]:
newListTPE = [] 
for i in dataFromEpohAccuracyTPE:
    newListTPE.append((i.cpu().detach().numpy()) )
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.plot(newList)
plt.savefig("TPETrain.png")

In [ ]:
newList2 = [] 
for i in dataFromEpohAccuracyTPE:
    newList2.append((i.cpu().detach().numpy()) )
plt.plot(newList2)


In [ ]:
#RandomSampler
netRandom = torch.nn.ModuleList()
complex_cur = 0
COMP_netRandom = []
SCR_netRandom = []

dataFromEpohAccuracyRandom = []
dataFromEpohLossRandom = []

netRandom.append(Cell_try_2(M=9, D=zz[0], Poly_order=8))
netRandom.append(Cell_try_2(M=5, D=zz[1], Poly_order=8))
netRandom.append(Cell_try_2(M=7, D=zz[2], Poly_order=7))
netRandom.append(Cell_try_2(M=5, D=zz[3], Poly_order=9))
netRandom.append(Cell_try_2(M=3, D=zz[4], Poly_order=5))



netRandom = netRandom.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netRandom.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netRandom.train()

train( train_queue, valid_queue, netRandom, loss_fn, optimizer,20,scheduler,save_flag=False,  dataFromEpohAccuracy = dataFromEpohAccuracyRandom, dataFromEpohLoss =dataFromEpohLossRandom )

loss_cur, accuracy_cur = eval_model(valid_queue, netRandom, loss_fn)
score_cur = accuracy_cur.item()

COMP_netRandom.append(complex_cur)
SCR_netRandom.append(score_cur)

print( score_cur)


In [ ]:
newListRandom = [] 
for i in dataFromEpohAccuracyRandom:
    newListRandom.append((i.cpu().detach().numpy()) )
newListRandom = sorted(newListRandom, reverse=True)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.plot(newListRandom)
plt.savefig("RandomTrain.png")

In [ ]:
#QMCSampler
netQMC = torch.nn.ModuleList()
complex_cur = 0
COMP_netQMC = []
SCR_netQMC = []


dataFromEpohAccuracyQMC = []
dataFromEpohLossQMC = []


netQMC.append(Cell_try_2(M=9, D=zz[0], Poly_order=6))
netQMC.append(Cell_try_2(M=7, D=zz[1], Poly_order=6))
netQMC.append(Cell_try_2(M=9, D=zz[2], Poly_order=8))
netQMC.append(Cell_try_2(M=7, D=zz[3], Poly_order=9))
netQMC.append(Cell_try_2(M=5, D=zz[4], Poly_order=7))



netQMC = netQMC.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netQMC.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netQMC.train()

train( train_queue, valid_queue, netQMC, loss_fn, optimizer,20,scheduler,save_flag=False,  dataFromEpohAccuracy = dataFromEpohAccuracyQMC, dataFromEpohLoss =dataFromEpohLossQMC )

loss_cur, accuracy_cur = eval_model(valid_queue, netQMC, loss_fn)
score_cur = accuracy_cur.item()

COMP_netQMC.append(complex_cur)
SCR_netQMC.append(score_cur)

print( score_cur)


In [ ]:
newListQMC = [] 
for i in dataFromEpohAccuracyQMC:
    newListQMC.append((i.cpu().detach().numpy()) )
newListQMC = sorted(newListQMC, reverse=True)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.plot(newListQMC)
plt.savefig("QMCTrain.png")

In [ ]:
#NSGAIISampler
netNSGAII = torch.nn.ModuleList()
complex_cur = 0
COMP_netNSGAII = []
SCR_netNSGAII = []

dataFromEpohAccuracyNSGAII = []
dataFromEpohLossNSGAII = []


netNSGAII.append(Cell_try_2(M=7, D=zz[0], Poly_order=8))
netNSGAII.append(Cell_try_2(M=5, D=zz[1], Poly_order=7))
netNSGAII.append(Cell_try_2(M=5, D=zz[2], Poly_order=8))
netNSGAII.append(Cell_try_2(M=7, D=zz[3], Poly_order=8))
netNSGAII.append(Cell_try_2(M=5, D=zz[4], Poly_order=5))



netNSGAII = netNSGAII.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netNSGAII.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netNSGAII.train()

train( train_queue, valid_queue, netNSGAII, loss_fn, optimizer,20,scheduler,save_flag=False, dataFromEpohAccuracy = dataFromEpohAccuracyNSGAII, dataFromEpohLoss =dataFromEpohLossNSGAII)

loss_cur, accuracy_cur = eval_model(valid_queue, netNSGAII, loss_fn)
score_cur = accuracy_cur.item()

COMP_netNSGAII.append(complex_cur)
SCR_netNSGAII.append(score_cur)

print( score_cur)



In [ ]:
newListQMC = [] 
for i in dataFromEpohAccuracyNSGAII:
    newListQMC.append((i.cpu().detach().numpy()) )
plt.xlabel('Iteration')
plt.ylabel('Loss')
newListNSGAII = newListQMC
plt.plot(newListNSGAII)

plt.savefig("NSGAIITrain.png")

In [ ]:
#CMA-EASampler

netCMA_EA = torch.nn.ModuleList()
complex_cur = 0
COMP_netCMA_EA = []
SCR_netCMA_EA = []


dataFromEpohAccuracyCMA = []
dataFromEpohLossCMA = []

netCMA_EA.append(Cell_try_2(M=5, D=zz[0], Poly_order=6))
netCMA_EA.append(Cell_try_2(M=7, D=zz[1], Poly_order=6))
netCMA_EA.append(Cell_try_2(M=7, D=zz[2], Poly_order=7))
netCMA_EA.append(Cell_try_2(M=7, D=zz[3], Poly_order=6))
netCMA_EA.append(Cell_try_2(M=7, D=zz[4], Poly_order=8))



netCMA_EA = netCMA_EA.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netCMA_EA.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netCMA_EA.train()

train( train_queue, valid_queue, netCMA_EA, loss_fn, optimizer,20,scheduler,save_flag=False,  dataFromEpohAccuracy = dataFromEpohAccuracyCMA, dataFromEpohLoss =dataFromEpohLossCMA)

loss_cur, accuracy_cur = eval_model(valid_queue, netCMA_EA, loss_fn)
score_cur = accuracy_cur.item()

COMP_netCMA_EA.append(complex_cur)
SCR_netCMA_EA.append(score_cur)

print( score_cur)



In [ ]:
newListQMC = [] 
for i in dataFromEpohAccuracyCMA:
    newListQMC.append((i.cpu().detach().numpy()) )
newListQMC = sorted(newListQMC, reverse=True)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title("CMA-E")
newListCMA = newListQMC
plt.plot(newListCMA)
plt.savefig("CMATrain.png")

In [ ]:
#Huge

netHuge = torch.nn.ModuleList()
complex_cur = 0
COMP_netHuge = []
SCR_netHuge = []

dataFromEpohAccuracyHuge = []
dataFromEpohLossHuge = []


netHuge.append(Cell_try_2(M=9, D=zz[0], Poly_order=9))
netHuge.append(Cell_try_2(M=9, D=zz[1], Poly_order=9))
netHuge.append(Cell_try_2(M=9, D=zz[2], Poly_order=9))
netHuge.append(Cell_try_2(M=9, D=zz[3], Poly_order=9))
netHuge.append(Cell_try_2(M=9, D=zz[4], Poly_order=9))



netHuge = netHuge.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netHuge.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netHuge.train()

train( train_queue, valid_queue, netHuge, loss_fn, optimizer,20,scheduler,save_flag=False, dataFromEpohAccuracy = dataFromEpohAccuracyHuge, dataFromEpohLoss =dataFromEpohLossHuge)

loss_cur, accuracy_cur = eval_model(valid_queue, netHuge, loss_fn)
score_cur = accuracy_cur.item()

COMP_netHuge.append(complex_cur)
SCR_netHuge.append(score_cur)

print( score_cur)


In [ ]:
newListHuge = [] 
for i in dataFromEpohAccuracyHuge:
    newListQMC.append((i.cpu().detach().numpy()) )
newListQMC = sorted(newListQMC, reverse=True)
plt.xlabel('Iteration')
plt.ylabel('Loss')
newListHuje = newListQMC
plt.plot(newListHuje)
plt.savefig("HugeTrain.png")

In [ ]:
#SmallModel

netSmall = torch.nn.ModuleList()
complex_cur = 0
COMP_netSmall = []
SCR_netSmall = []


dataFromEpohAccuracySmall = []
dataFromEpohLossSmall = []


netSmall.append(Cell_try_2(M=3, D=zz[0], Poly_order=4))
netSmall.append(Cell_try_2(M=3, D=zz[1], Poly_order=4))
netSmall.append(Cell_try_2(M=3, D=zz[2], Poly_order=4))
netSmall.append(Cell_try_2(M=3, D=zz[3], Poly_order=4))
netSmall.append(Cell_try_2(M=3, D=zz[4], Poly_order=4))



netSmall = netSmall.to(torch.device('cpu'))
optimizer = torch.optim.Adam(netSmall.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
netSmall.train()

train( train_queue, valid_queue, netSmall, loss_fn, optimizer,20,scheduler,save_flag=False, dataFromEpohAccuracy = dataFromEpohAccuracySmall, dataFromEpohLoss =dataFromEpohLossSmall)

loss_cur, accuracy_cur = eval_model(valid_queue, netSmall, loss_fn)
score_cur = accuracy_cur.item()

COMP_netSmall.append(complex_cur)
SCR_netSmall.append(score_cur)

print( score_cur)


In [ ]:
newListQMC = [] 
for i in dataFromEpohAccuracySmall:
    newListQMC.append((i.cpu().detach().numpy()) )
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.plot(newListQMC)
plt.savefig("SmallTrain.png")

In [ ]:
#UltraGraf
ultraList = [newList,  newListRandom,  newListQMC, newListNSGAII,  newListCMA,  newListHuje]
#count = 1 
#for i in ultraList:
#    plt.plot (i, label = str(count))
#    count += 1 
#plt.legend()
newListCMA = newListCMA[:20]
plt.plot(newList, label = "TPE")
plt.plot(newListRandom, label = "Random")
plt.plot(newListQMC, label = "QMC")
plt.plot(newListNSGAII, label = "GA")
plt.plot(newListCMA, label = "CMA-E")
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.savefig("grafShod.png")
plt.show()

In [ ]:
#outTPE = sum(list(map( lambda n: n(input_batch),  net)))
outRandom = sum(list(map( lambda n: n(input_batch),  netRandom)))
outQMC = sum(list(map( lambda n: n(input_batch),  netQMC)))
outNSGAII = sum(list(map( lambda n: n(input_batch),  netNSGAII)))
outCMA_EA = sum(list(map( lambda n: n(input_batch),  netCMA_EA)))
outHuge = sum(list(map( lambda n: n(input_batch),  netHuge)))
outTPE = sum(list(map( lambda n: n(input_batch),  netTPE)))
outSmall = sum(list(map( lambda n: n(input_batch),  netSmall)))

print (outSmall)
torch.save(outSmall, 'outSmall.pt')

In [ ]:
torch.load(outQMC, 'outQMC.pt')
torch.save(outNSGAII, 'outNSGAII.pt')
torch.save(outCMA_EA, 'outCMA_EA.pt')
torch.save(outHuge, 'outHuge.pt')



In [ ]:
outQMC = torch.load('outQMC.pt')
outNSGAII = torch.load( 'outNSGAII.pt')
outCMA_EA = torch.load( 'outCMA_EA.pt')
outHuge = torch.load( 'outHuge.pt')
outSmall = torch.load( 'outSmall.pt')
outTPE = torch.load( 'outTPE.pt')
outRandom = torch.load( 'outRandom.pt')

In [ ]:


net.eval()

#input_signal = Variable(valid_queue[:,:,:1],requires_grad=False).permute(2,1,0).cpu()

#out_228 = net(valid_queue)

for step, (valid) in enumerate(valid_queue):
    net.eval()
    print("Valid: ", type(valid)) 
    print( "Valid_queue: ", type(valid_queue))
    
    input_batch = Variable(valid[:,:,:1],requires_grad=False).permute(2,1,0).cpu()
    desired = Variable(valid[:,:,1:],requires_grad=False).permute(2,1,0).cpu()
    #out = model.forward(input_batch)
    outTPE = sum(list(map( lambda n: n(input_batch),  netTPE)))
    outRandom = sum(list(map( lambda n: n(input_batch),  netRandom)))
    outQMC = sum(list(map( lambda n: n(input_batch),  netQMC)))
    outNSGAII = sum(list(map( lambda n: n(input_batch),  netNSGAII)))
    outCMA_EA = sum(list(map( lambda n: n(input_batch),  netCMA_EA)))

    
    plt.psd(((desired-outTPE).detach().cpu()[0,0,:]+1j*(desired-outTPE).detach().cpu()[0,1,:]),NFFT=2048, label='Error reference', ls='--', c='m')
    plt.psd(((outTPE).detach().cpu()[0,0,:]+1j*(outTPE).detach().cpu()[0,1,:]),NFFT=2048, label='1 model  out', ls='--', c='g')


    plt.psd((input_batch.detach().cpu()[:,0,0]+1j*input_batch.detach().cpu()[:,1,0]),NFFT=2048,label=' Signal',c='r')
    plt.psd(d.reshape(-1,),NFFT=2048,label='Desired',color='orange')
    plt.psd(((desired).detach().cpu()[0,0,:]+1j*(desired).detach().cpu()[0,1,:]),NFFT=2048, label='1 model desired', ls='--', c='pink')

    plt.legend()
    #loss=criterion(out,desired)
    #draw_spectrum(input_batch,desired,out)
    accuracy = NMSE(input_batch, outTPE - desired)


    

In [ ]:
for step, (valid) in enumerate(valid_queue):
    desired = Variable(valid[:,:,1:],requires_grad=False).permute(2,1,0).cpu()


In [ ]:
#model.eval()

print (type(outTPE))
print (type(input_batch))


In [ ]:
#type(input_batch)
plt.psd(X.detach().cpu()[:,0,0]+1j*X.detach().cpu()[:,1,0],NFFT=2048, label = "X")
plt.psd(input_batch.detach().numpy().reshape(-1,),NFFT=2048, label = "input_batch")

plt.psd(out.detach().numpy().reshape(-1,),NFFT=2048, label = "out")
plt.psd(out322.detach().numpy().reshape(-1,),NFFT=2048, label = "out322")

#plt.psd(input_batch.detach().numpy().reshape(-1,),NFFT=2048, label = "input_batch")
plt.psd(desired.detach().numpy().reshape(-1,),NFFT=2048, label = "desired")
plt.psd((out - desired).detach().numpy().reshape(-1,),NFFT=2048, label = "minus")
plt.psd(d.reshape(-1,),NFFT=2048, label = "d",ls='--')
plt.psd(ref.reshape(-1,),NFFT=2048, label = "ref",ls='--')
#plt.psd(D.reshape(-1,),NFFT=2048, label = "d")
#plt.psd(X.reshape(-1,),NFFT=2048, label = "x")
#plt.psd(ref.reshape(-1,),NFFT=2048, label = "ref")
plt.legend()


In [ ]:
%matplotlib inline

In [ ]:

#from Data.utilits.draw_functions import plot_psd_complex2
X2 = X
X2 = torch.permute(X2, (2, 1, 0))
justnumber = 4
plot_psd_complex3 ({ 
                    "Signal" : ((X.detach().cpu()[:,0,0]+1j*X.detach().cpu()[:,1,0]) / 2**15 )*justnumber,
                    ("Desired"):(((desired).detach().cpu()[0,0,:]+1j*(desired).detach().cpu()[0,1,:])/2**15)*justnumber,
                   "TPE" : (((desired - outTPE + X2  ).detach().cpu()[0,0,:]+1j*(desired - outTPE + X2).detach().cpu()[0,1,:])/2**15)*justnumber, 
                   "Random":((desired - outRandom + X2 ).detach().cpu()[0,0,:]/2**15+1j*(desired - outRandom + X2 ).detach().cpu()[0,1,:]/2**15)*justnumber,
                  "NSGAII":((desired - outNSGAII + X2  ).detach().cpu()[0,0,:]/2**15+1j*(desired - outNSGAII + X2 ).detach().cpu()[0,1,:]/2**15)*justnumber,
                   "QMC":((desired - outQMC + X2 ).detach().cpu()[0,0,:]/2**15+1j*(desired - outQMC + X2).detach().cpu()[0,1,:]/2**15)*justnumber,
                  "CMA-EA":((desired - outCMA_EA + X2 ).detach().cpu()[0,0,:]/2**15+1j*(desired - outCMA_EA + X2).detach().cpu()[0,1,:]/2**15)*justnumber, 
                   "Huge model" :  ((desired - outHuge + X2 ).detach().cpu()[0,0,:]/2**15+1j*(desired - outHuge + X2).detach().cpu()[0,1,:]/2**15)*justnumber,                                                                                }, fs=4*122.88,
                 )
plt.legend(prop={'size': 7})
plt.savefig("PowerSP_Freq.png")


In [ ]:
NMSE(outNSGAII , outTPE)
NMSE(outNSGAII , outTPE)

In [ ]:
from Data.utilits import ACPR_calc

error = (desired - outNSGAII + X2  ).detach().cpu()[0,0,:]+1j*(desired - outNSGAII + X2).detach().cpu()[0,1,:]
Xcom = (X.detach().cpu()[:,0,0]+1j*X.detach().cpu()[:,1,0])
print(error)
acpr = ACPR_calc(Xcom.cpu().detach().numpy().reshape(-1)+
            (error ).cpu().detach().numpy().reshape(-1),4*122.88e6,  10e6,-72e6,52e6,20e6,20e6)
print('ACPR={}'.format(acpr))

In [ ]:
plot_psd_complex2 ({ 
    #"Desired":(((torch.nn.functional.normalize(desired,dim = 1)).detach().cpu()[0,0,:]+1j*((torch.nn.functional.normalize(desired,dim = 1))).detach().cpu()[0,1,:])/2**15)+justnumber,
                  ("Signal", '--', 0.3, "_",12 ) : ( torch.nn.functional.normalize(X,dim = 1).detach().cpu()[:,0,0]+1j*torch.nn.functional.normalize(X,dim = 1).detach().cpu()[:,1,0]) / 2**15 ,
                    ("Desired"):(((torch.nn.functional.normalize(desired,dim = 1) ).detach().cpu()[0,0,:]+1j*(torch.nn.functional.normalize(desired,dim = 1)).detach().cpu()[0,1,:])/2**15)+justnumber,
                    "TPE" : (((TPE2  ).detach().cpu()[0,0,:]+1j*( TPE2).detach().cpu()[0,1,:])/2**15)+justnumber, 
                   "Random":(( Random2 ).detach().cpu()[0,0,:]/2**15+1j*( Random2 ).detach().cpu()[0,1,:]/2**15)+justnumber,
                  "NSGAII":(( NSGAII2  ).detach().cpu()[0,0,:]/2**15+1j*( NSGAII2 ).detach().cpu()[0,1,:]/2**15)+justnumber,
                   "QMC":(( QMC2  ).detach().cpu()[0,0,:]/2**15+1j*( QMC2 ).detach().cpu()[0,1,:]/2**15)+justnumber,
                  "CMA-EA":(( CMA_EA2 ).detach().cpu()[0,0,:]/2**15+1j*( CMA_EA2).detach().cpu()[0,1,:]/2**15)+justnumber,

}, fs=4*122.88, name = "Graf1.png"
                 )


In [ ]:
TPE2 = outTPE
Random2 = outRandom
NSGAII2 = outNSGAII
QMC2 = outQMC
CMA_EA2 = outCMA_EA


XNorm = X
XNorm = torch.permute(XNorm, (2, 1, 0))
print (XNorm.shape)

TPE2 = (torch.nn.functional.normalize(desired,dim = 1) - torch.nn.functional.normalize(TPE2,dim = 1) ) +  XNorm 
Random2 = (torch.nn.functional.normalize(desired,dim = 1) - torch.nn.functional.normalize(Random2,dim = 1) ) +  XNorm 
NSGAII2 = (torch.nn.functional.normalize(desired,dim = 1) - torch.nn.functional.normalize(NSGAII2,dim = 1) ) +  XNorm 
QMC2 = (torch.nn.functional.normalize(desired,dim = 1) - torch.nn.functional.normalize(QMC2,dim = 1) ) +  XNorm 
CMA_EA2 = (torch.nn.functional.normalize(desired,dim = 1) - torch.nn.functional.normalize(CMA_EA2,dim = 1) ) +  XNorm 

print (TPE2.shape)

In [ ]:
def plot_psd_complex2(*signals, fs: float, N: int = 2048, window: str = 'hann',name: str = 'pic.png'):
    """Compute PSD of multiple signals using Welch method and plot them

    Parameters
    ----------
    signals :
        dictionary of signals to compute PSD,
        format:
        1st variant: {("name_1", ls, lw, marker, markersize): signal_1, ("name_2", ls, lw, marker, markersize): signal_2, ...}
        2nd variant: {"name_1": signal_1, "name_2": signal_2, ...}, format can be choosen separately for each plot
    fs : float              4*122.88
        sampling frequency [MHz]
    N : int, optional
        window size, by default 2048
    window : str, optional
        window type (windows are taken from `sp.signal.windows`), by default 'hann'
    """

    # defaults
    ls = '-'
    lw = 0.3
    marker = '.'
    markersize = 0.5
    color = ['blue','red','green','purple','orange', 'olive','gray','black']
    plt.figure()
    plt.xlabel('Freq [MHz]')
    plt.ylabel('Power Spectrum [dB]')
    for info, s in signals[0].items():
        if type(info) != tuple:
            info = (info, ls, lw, marker, markersize)

        # default detrend removes dc
        f_psd, signal_psd = sp.signal.welch(x=s, fs=fs, scaling='spectrum', return_onesided=False, detrend=False,
                                            window=sp.signal.get_window(window, N))
        arg_sort = np.argsort(f_psd)
        f_psd = np.roll(f_psd, N // 2, axis=0)
        signal_psd = np.roll(signal_psd, N // 2, axis=0)
        if c
        plt.plot(f_psd, 10 * np.log10(signal_psd), label=info[0], ls=info[1], lw=info[2], marker=info[3],
                 markersize=info[4])
    plt.xlim(-100, 100)
    #plt.ylim(-25, 5 )

    plt.legend(loc='lower center')
    plt.grid('both')
    #plt.savefig(name)

   # plt.show()

In [ ]:
def plot_psd_complex3(*signals, fs: float, N: int = 2048, window: str = 'hann',name: str = 'pic.png'):
    """Compute PSD of multiple signals using Welch method and plot them

    Parameters
    ----------
    signals :
        dictionary of signals to compute PSD,
        format:
        1st variant: {("name_1", ls, lw, marker, markersize): signal_1, ("name_2", ls, lw, marker, markersize): signal_2, ...}
        2nd variant: {"name_1": signal_1, "name_2": signal_2, ...}, format can be choosen separately for each plot
    fs : float              4*122.88
        sampling frequency [MHz]
    N : int, optional
        window size, by default 2048
    window : str, optional
        window type (windows are taken from `sp.signal.windows`), by default 'hann'
    """

    # defaults
    ls = '-'
    lw = 0.3
    marker = '.'
    markersize = 0.5

    plt.figure()
    plt.xlabel('Freq [MHz]')
    plt.ylabel('Power Spectrum [dB]')
    color = ['blue','red','green','purple','orange', 'olive','gray','black']
    countColor = 0
    for info, s in signals[0].items():
        if type(info) != tuple:
            info = (info, ls, lw, marker, markersize)

        # default detrend removes dc
        f_psd, signal_psd = sp.signal.welch(x=s, fs=fs, scaling='spectrum', return_onesided=False, detrend=False,
                                            window=sp.signal.get_window(window, N))
        arg_sort = np.argsort(f_psd)
        f_psd = np.roll(f_psd, N // 2, axis=0)
        signal_psd = np.roll(signal_psd, N // 2, axis=0)
        if countColor == 0:
            plt.plot(f_psd, 10 * np.log10(signal_psd)+114, label=info[0], ls=info[1], lw=2, marker=info[3],
                 markersize=info[4],c=color[countColor])
        else:
            plt.plot(f_psd, 10 * np.log10(signal_psd)+114, label=info[0], ls=info[1], lw=info[2], marker=info[3],
                 markersize=info[4],c=color[countColor])
        countColor += 1 
    plt.xlim(-150, 150)
    plt.ylim(-90, 5 )

    plt.legend(loc='lower center')
    plt.grid('both')
    #plt.savefig(name)

    #plt.show()

In [ ]:
print(complex_huge)
### params of the functional
score_huge = -33.88
score_min = -20.0
complex_huge = sum(ref_model['k']) + sum(ref_model['p'])
complex_min = 4 * 5 + 3 * 5

COMP_netHuge 
SCR_netHuge 

In [ ]:
print(SCR_netHuge) 